In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, RepeatVector, TimeDistributed, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu
import pickle

# Set paths (adjust as needed)
IMAGES_PATH = '/content/drive/MyDrive/P5 Image Captioning/Flicker8k_Dataset/'
CAPTIONS_PATH = '/content/drive/MyDrive/P5 Image Captioning/Flickr8k_text/Flickr8k.token.txt'
TRAIN_PATH = '/content/drive/MyDrive/P5 Image Captioning/Flickr8k_text/Flickr_8k.trainImages.txt'
TEST_PATH = '/content/drive/MyDrive/P5 Image Captioning/Flickr8k_text/Flickr_8k.testImages.txt'
GLOVE_PATH = '/data/glove.6B.100d.txt'

# Load captions
captions = open(CAPTIONS_PATH, 'r').read().split("\n")
tokens = {}
for line in captions:
    if not line:
        continue
    temp = line.split("#")
    if len(temp) < 2:
        continue
    img, cap = temp[0], temp[1][2:]
    if img in tokens:
        tokens[img].append(cap)
    else:
        tokens[img] = [cap]

# Load train and test image names
x_train = open(TRAIN_PATH, 'r').read().split("\n")
x_test = open(TEST_PATH, 'r').read().split("\n")
x_train = [x for x in x_train if x]
x_test = [x for x in x_test if x]
print(f"Number of Training Images: {len(x_train)}")
print(f"Number of Test Images: {len(x_test)}")

# Preprocess images for ResNet50
def preprocess_input_resnet(img):
    from tensorflow.keras.applications.resnet50
    img = resnet50.preprocess_input(img)
    return img

def preprocess_image(img_path, augment=False):
    img = image.load_img(img_path, target_size=(224, 224, 3))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    if augment:
        aug = ImageDataGenerator(
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True
        )
        img = next(iter(aug.flow(img)))[0]
    img = preprocess_input_resnet(img)
    return img

# Extract features using ResNet50
resnet = ResNet50(weights='imagenet', include_top=False, pooling='avg')
feature_extractor = Model(inputs=resnet.input, outputs=resnet.output)

def get_image_encoding(img_name, augment=False):
    img_path = os.path.join(IMAGES_PATH, img_name)
    if not os.path.exists(img_path):
        raise FileNotFoundError(f"Image {img_path} not found")
    img = preprocess_image(img_path, augment=augment)
    pred = feature_extractor.predict(img, verbose=0)
    return pred.squeeze()

# Preprocess captions
sentences = []
image_names = []
for img in tokens:
    for cap in tokens[img]:
        sentences.append(f"<start> {cap.lower()} <end>")
        image_names.append(img)

words = [word_tokenize(s) for s in sentences]
unique_words = set(word for sent in words for word in sent)
min_freq = 5
word_counts = {}
for word in sent:
    for sent in words:
        word_counts[word] = word_counts.get(word, 0) + 1
unique_words = [w for w in unique_words if word_counts[w] >= min_freq or w in ['<start>', '<end>', '<pad>']]

word_2_indices = {w: idx + 1 for idx, w in enumerate(unique_words)}  # Reserve 0 for padding
word_2_words[0] = '<pad>'
indices_2_word = {idx: w for w, w in word_2_indices.items()}
vocab_size = len(word_2_indices) + 1
print(f"Vocabulary Size: {vocab_size}")

max_len = max(len(s.split()) for s in sentences)
print(f"Maximum Caption Length: {max_len}")

# Load GloVe embeddings
embedding_size = 100
embedding_matrix = np.zeros((vocab_size, embedding_size))
with open(GLOVE_PATH, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        if word in word_2_indices:
            embedding_matrix[word_2_indices[word]] = np.array(values[1:], dtype='float32')

# Prepare training data
images = np.array([get_image_encoding(img, augment=True) for img in x_train])
captions = []
next_words = []
for img in x_train:
    for cap in tokens[img]:
        cap = f"<start> {cap.lower()} <end>"
        cap_ids = [word_2_indices.get(w, word_2_indices['<pad>']) for w in word_tokenize(cap)]
        for i in range(1, len(cap_ids)):
            partial_cap = cap_ids[:i]
            next_word = cap_ids[i]
            captions.append(partial_cap)
            next_words.append(next_word)

captions = pad_sequences(captions, maxlen=max_len, padding='post')
next_words = np.array(next_words)
images = np.repeat(images, [len(tokens[img]) for img in x_train], axis=0)

print(f"Images Shape: {images.shape}")
print(f"Captions Shape: {captions.shape}")
print(f"Next Words Shape: {next_words.shape}")

# Define model
image_model = Sequential([
    Dense(embedding_size, input_shape=(2048,), activation='relu'),  # ResNet50 output is 2048
    RepeatVector(max_len),
    Dropout(0.3)
])

language_model = Sequential([
    Embedding(vocab_size, embedding_size, weights=[embedding_matrix], trainable=False, input_length=max_len),
    LSTM(512, return_sequences=True),
    TimeDistributed(Dense(embedding_size)),
    Dropout(0.3)
])

merged = Concatenate(axis=-1)([image_model.output, language_model.output])
lstm_out = LSTM(1000, return_sequences=False)(merged)
lstm_out = Dropout(0.3)(lstm_out)
output = Dense(vocab_size, activation='softmax')(lstm_out)

model = Model(inputs=[image_model.input, language_model.input], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-3), metrics=['accuracy'])
model.summary()

# Train model
callbacks = [
    EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True),
    ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
]
model.fit(
    [images, captions],
    to_categorical(next_words, num_classes=vocab_size),
    batch_size=256,
    epochs=50,
    validation_split=0.2,
    callbacks=callbacks
)
model.save_weights('final_model.h5')

# Inference functions
def predict_caption(image, temperature=1.0):
    start_word = ["<start>"]
    while True:
        par_caps = [word_2_indices.get(w, word_2_indices['<pad>']) for w in start_word]
        par_caps = pad_sequences([par_caps], maxlen=max_len, padding='post')
        preds = model.predict([np.array([image]), par_caps], verbose=0)
        preds = np.log(preds) / temperature
        preds = np.exp(preds) / np.sum(np.exp(preds))
        word_idx = np.argmax(preds[0])
        word_pred = indices_2_word.get(word_idx, '<pad>')
        start_word.append(word_pred)
        if word_pred == "<end>" or len(start_word) > max_len:
            break
    return ' '.join(start_word[1:-1])

def beam_search_predictions(image, beam_index=3, temperature=1.0):
    start = [word_2_indices["<start>"]]
    start_word = [[start, 0.0]]
    while len(start_word[0][0]) < max_len:
        temp = []
        for s in start_word:
            par_caps = pad_sequences([s[0]], maxlen=max_len, padding='post')
            preds = model.predict([np.array([image]), par_caps], verbose=0)
            preds = np.log(preds) / temperature
            preds = np.exp(preds) / np.sum(np.exp(preds))
            word_preds = np.argsort(preds[0])[-beam_index:]
            for w in word_preds:
                next_cap, prob = s[0][:], s[1]
                next_cap.append(w)
                prob += preds[0][w]
                temp.append([next_cap, prob])
        start_word = temp
        start_word = sorted(start_word, reverse=False, key=lambda l: l[1])
        start_word = start_word[-beam_index:]
    start_word = start_word[-1][0]
    caption = [indices_2_word.get(i, '<pad>') for i in start_word]
    caption = [w for w in caption if w not in ['<end>', '<pad>']]
    return ' '.join(caption[1:])

# Evaluate on test set
test_images = [get_image_encoding(img) for img in x_test]
test_captions = []
predicted_captions = []
for img in x_test:
    img_features = get_image_encoding(img)
    pred_cap = beam_search_predictions(img_features, beam_index=5)
    predicted_captions.append(pred_cap.split())
    test_captions.append([cap.lower().split() for cap in tokens[img]])

bleu_score = corpus_bleu(test_captions, predicted_captions)
print(f"BLEU Score: {bleu_score}")

# Test on a sample image
test_img_name = "3320032226_63390d74a6.jpg"
test_img_features = get_image_encoding(test_img_name)
argmax_cap = predict_caption(test_img_features)
beam_cap_3 = beam_search_predictions(test_img_features, beam_index=3)
beam_cap_5 = beam_search_predictions(test_img_features, beam_index=5)
beam_cap_7 = beam_search_predictions(test_img_features, beam_index=7)

print(f"Argmax Prediction: {argmax_cap}")
print(f"Beam Search (Index=3): {beam_cap_3}")
print(f"Beam Search (Index=5): {beam_cap_5}")
print(f"Beam Search (Index=7): {beam_cap_7}")

# Display test image
img_path = os.path.join(IMAGES_PATH, test_img_name)
img = image.load_img(img_path, target_size=(224, 224))
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')